<a href="https://colab.research.google.com/github/GURJEW/tasks/blob/main/%D0%93%D1%83%D1%80%D1%8C%D0%B5%D0%B2_%D0%92%D0%B0%D1%81%D0%B8%D0%BB%D0%B8%D0%B9_%D0%A2%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%BE%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Task 1

Имеется база со статистикой баскетбольных матчей (https://drive.google.com/file/d/1TGiwfCMtryCLy0acYgsRtRd-QHLUhlpA/view?usp=sharing). Напишите нейронную сеть, которая будет предсказывать результат матча. Можно выбрать один из вариантов (будем плюсом реализовать оба).



1.   нейронка, предсказывающая результат в произвольный момент времени (на вход подается количество мячей, забитых каждой из команд, и время прошедшее с начала матча. На выходе - суммарное количество мячей, забитых обеими командами в конце матча.)
2.   нейронка, предсказывающая результат после второго периода (на вход приходят данные за сыгранные два периода. На выходе - суммарное количество мячей, забитых обеими командами в конце матча).



##Imports

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import os
from IPython.display import clear_output
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/Colab Notebooks/AIU/TEST')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##Defs

###history_plot

In [ ]:
def history_plot(history):
    epochs = len(history['loss'])
    history.pop('lr', None)
    fig = make_subplots(rows=1, cols=2)
    for score in history.keys():
        fig.add_trace(
            go.Scatter(
                x=[i + 1 for i in range(epochs)],
                y=history[score],
                name=score
                ),
                row=1,
                col=[1, 2]['mae' in score]
            )
    fig.update_layout(
        legend_orientation='h',
        margin={'l': 0, 'r': 40, 't': 40, 'b': 0},
        title='History'
        )
    fig.update_xaxes(title='epochs')
    fig.update_yaxes(title='loss (MSE)', row=1, col=1)
    fig.update_yaxes(title='MAE', row=1, col=2)
    fig.update_traces(hoverinfo='all', hovertemplate="epoch: %{x}<br>error: %{y}")
    fig.show()

###box_plot

In [ ]:
def box_plot(data_frame, color, title, x='subset', y='mae',):
    fig = px.box(data_frame=data_frame, x=x, y=y, color=color)
    fig.update_layout(title=title, margin=dict(l=0, r=200, t=40, b=0))
    fig.show()

###to_log

In [ ]:
def to_log(path, record, new_log=False):
    if new_log:
        log = pd.DataFrame(columns=record.keys())
    else:    
        try:
            log = pd.read_csv(path, index_col=0)
        except:
            log = pd.DataFrame(columns=record.keys())
    log.loc[len(log)] = record
    log.to_csv(path)

###scaler

In [ ]:
def scaler(x, y):
    x = MinMaxScaler().fit_transform(x)
    y = np.array(y).reshape(-1, 1)
    y_scaler = MinMaxScaler()
    y_scaler.fit(y)
    return x, y_scaler.transform(y), y_scaler

###research

In [ ]:
def research(
    data,
    mode,
    n_trial=10,
    outlier_threshold=0,
    hidden_dim=20,
    depth=1,
    activation='relu',
    learning_rate=0.001,
    batch_size=128
    ):

    for n in range(n_trial):
        if mode == 'time': # данные по времени, как есть
            x, y, y_scaler = scaler(
                data[['team_1', 'team_2', 'time']],
                data.final_count
                )
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
        elif mode == 'sum': # так как очки команд сильно коррелируются друг с другом, попробуем сократить размерность
            x, y, y_scaler = scaler(
                data[['current_count', 'time']],
                data.final_count
            )
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
        elif mode == 'time_2': # сделаем аугментацуию поменяв местами команды
            x, y, y_scaler = scaler(
                np.vstack(
                [data[['team_1', 'team_2', 'time']].to_numpy(), data[['team_2', 'team_1', 'time']].to_numpy()]
                ),
                np.array(data.final_count.to_list() * 2)
            )
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
        elif mode == 'quarter_1': # предсказания по результатам первой четверти
            x, y, y_scaler = scaler(data[1], data.final_count)
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
        elif mode == 'quarter_2': # предсказания по результатам второй четверти
            x, y, y_scaler = scaler(data[2], data.final_count)
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
        elif mode == 'quarter_3': # предсказания по результатам третьей четверти
            x, y, y_scaler = scaler(data[3], data.final_count)
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
        elif mode == 'quarter_12': # предсказания по результатам первой и второй четверти
            x, y, y_scaler = scaler(data[[1, 2]], data.final_count)
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
        elif mode == 'quarter_23': # предсказания по результатам второй и третьей четверти
            x, y, y_scaler = scaler(data[[2, 3]], data.final_count)
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
        elif mode == 'quarter_123': # предсказания по результатам первой, второй и третьей четверти
            x, y, y_scaler = scaler(data[[1, 2, 3]], data.final_count)
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

        #удаляем выбросы
        x_train = x_train[y_scaler.inverse_transform(y_train).flatten() > outlier_threshold]
        y_train = y_train[y_scaler.inverse_transform(y_train).flatten() > outlier_threshold]

        inputs = Input(x_train.shape[-1])
        hiddens = inputs
        for _ in range(depth):
            hiddens = Dense(hidden_dim, activation)(hiddens)
        outputs = Dense(1, 'linear')(hiddens)

        model = Model(inputs, outputs)

        model.compile(
            optimizer=Adam(learning_rate),
            loss='mse',
            metrics=['mae']
        )

        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)
        early_stopping = EarlyStopping(restore_best_weights=True, patience=7)
        history = model.fit(
            x=x_train,
            y=y_train,
            batch_size=batch_size,
            epochs=2000,
            validation_split=0.2,
            callbacks=[reduce_lr, early_stopping]
        )
        clear_output()
        history_plot(history.history)
        
        train_loss, train_mae = model.evaluate(x_train, y_train, verbose=0)
        test_loss, test_mae = model.evaluate(x_test, y_test, verbose=0)
        
        print(f'\nСредняя ошибка на тестовой выборке: {round(y_scaler.inverse_transform([[test_mae]])[0][0], 2)} очков.')
        
        for subset in ['train', 'test']:
            record = {
                'subset': subset,
                'mode': mode,
                'outlier_threshold': outlier_threshold,
                'hidden_dim': hidden_dim,
                'depth': depth,
                'activation': activation,
                'batch_size': batch_size,
                'loss': eval(f'round({subset}_loss, 4)'),
                'mae': eval(f'round(y_scaler.inverse_transform([[{subset}_mae]])[0][0], 2)')
            }
            to_log('log.csv', record)

##EDA

Предварительный анализ данных

###Подготовка данных для предсказания по времени

In [ ]:
time_data = pd.read_csv(
    'basketball.csv',
    encoding='cp1251',
    sep=';',
    index_col=0,
    usecols=[0, 2, 3, 4, 8, 9] # выбираем нужные признаки
    )
time_data.columns = ['game', 'team_1', 'team_2', 'final_count', 'time'] # делаем удобные имена
time_data['current_count'] = time_data.team_1 + time_data.team_2 # добавляем поле текущих очков
time_data['quarter'] = time_data.time.apply(lambda x: x // 600 + 1) # определяем четверть
time_data.head()

,game,team_1,team_2,final_count,time,current_count,quarter
0,4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,0.0,81,90.0,2.0,1.0
1,4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,81,105.0,4.0,1.0
2,4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,81,120.0,4.0,1.0
3,4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,81,150.0,4.0,1.0
4,4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2,2.0,81,180.0,4.0,1.0


In [ ]:
(time_data.groupby('game').agg({'time': 'max'}) / 600).round().time.value_counts() # посмотрим на количество четвертей

4.0    414
1.0      7
5.0      7
2.0      3
3.0      2
0.0      1
6.0      1
Name: time, dtype: int64

Если 5 и 6 четвертей можно объяснить дополнительным временем, то меньшее количество вызывает вопросы.

In [ ]:
time_data.game.value_counts().tail(10) # посмотрим количество записей каждой игры

4169331 Литва. Мужчины. LKL. Плей-офф. 1/4 финала(Жальгирис - Пено Жвайгждес)\n                                 27
4085297 Россия. Женщины. Суперлига. 2-й дивизион. Плей-офф. 1/2 финала. Оренбург(Надежда-2 - Энергия-2)\n       21
4146967 Литва. Мужчины. NKL. Плей-офф. Матч за 3-е место(Петрохема - Паланга)\n                                 20
4085327 Нордеа Юхислиига. Женщины. Плей-офф. 1/8 финала. Ответные матчи(ТТТ Рига-мол - Латвияс Университи)\n    17
4184627 Португалия. Мужчины. Про-лига. Плей-офф. 1/2 финала(Атлетико-Экспрессо - Тершейра)\n                     5
4146967 Литва. Мужчины. NKL. Плей-офф. Матч за 3-е место. Паланга(Петрохема - Паланга)\n                         5
4148645 Венесуэла. Мужчины. LPB. Плей-офф. 1/4 финала(Маринос - Гигантес)\n                                      3
4170266 Чехия. Мужчины. NBL. Плей-офф. 1/2 финала(Простеев - Дечин)\n                                            3
4163325 Канада. Мужчины. NBL(Оринджевилл Эйс - Лондон Лайтнинг)\n               

Настораживает малое количесьво записей. Вероятно там будут выбросы.

In [ ]:
outliers = list(time_data.game.value_counts().index[-6:]) # берём 6 нижних строчек

In [ ]:
time_data.query('game == @outliers') # и смотрим содержание игр

,game,team_1,team_2,final_count,time,current_count,quarter
5607,4170266 Чехия. Мужчины. NBL. Плей-офф. 1/2 фин...,73,64.0,138,2385.0,137.0,4.0
5608,4170266 Чехия. Мужчины. NBL. Плей-офф. 1/2 фин...,74,64.0,138,2390.0,138.0,4.0
5609,4170266 Чехия. Мужчины. NBL. Плей-офф. 1/2 фин...,74,64.0,138,2400.0,138.0,5.0
21490,4184627 Португалия. Мужчины. Про-лига. Плей-оф...,60,52.0,118,2370.0,112.0,4.0
21491,4184627 Португалия. Мужчины. Про-лига. Плей-оф...,62,52.0,118,2385.0,114.0,4.0
21492,4184627 Португалия. Мужчины. Про-лига. Плей-оф...,62,52.0,118,2390.0,114.0,4.0
21493,4184627 Португалия. Мужчины. Про-лига. Плей-оф...,65,52.0,118,2390.0,117.0,4.0
21494,4184627 Португалия. Мужчины. Про-лига. Плей-оф...,65,53.0,118,2390.0,118.0,4.0
55522,4146967 Литва. Мужчины. NKL. Плей-офф. Матч за...,15,12.0,44,435.0,27.0,1.0
55523,4146967 Литва. Мужчины. NKL. Плей-офф. Матч за...,15,18.0,44,480.0,33.0,1.0


Некоторые данные вполне релевантны, несмотря на неполное представление игры. А те, где итоговая сумма очков слишком мала, определённо являются выбросами.

In [ ]:
time_data.final_count.value_counts().sort_index()[:10] # посмотрим на минимальное финальное количество очков

2      1
6      3
17    17
24    20
29    32
44     5
56    83
62    46
65    54
74    56
Name: final_count, dtype: int64

Меньше 40-50 можно считать выбросами.

In [ ]:
time_data.game.nunique() # 

435

In [ ]:
time_data.game.apply(lambda x: x.split()[0]).nunique() # 

434

В данных ошибка: есть повторение номера игры.

In [ ]:
time_data.game.apply(lambda x: ' '.join(x.split()[1:])).nunique() # 

423

Есть и повторения описаний при разных номерах.

In [ ]:
time_data['number'] = time_data.game.apply(lambda x: x.split()[0]) # создаём поле с номером игры

In [ ]:
time_data.game.loc[time_data.number == pd.Series(time_data.game.unique()) \ # находим дубль
              .apply(lambda x: x.split()[0]).value_counts().index[0]].unique()

array(['4146967 Литва. Мужчины. NKL. Плей-офф. Матч за 3-е место(Петрохема - Паланга)\n',
       '4146967 Литва. Мужчины. NKL. Плей-офф. Матч за 3-е место. Паланга(Петрохема - Паланга)\n'],
      dtype=object)

Действительно эти данные стоит объединить.

In [ ]:
time_data['description'] = time_data.game.apply(lambda x: ' '.join(x.split()[1:])) # создаём поле описания игры

In [ ]:
time_data.game.loc[time_data.description == pd.Series(time_data.game.unique()) \ # находим дубль
              .apply(lambda x: ' '.join(x.split()[1:])).value_counts().index[0]].unique()

array(['4171925 Литва. Мужчины. LKL. Плей-офф. 1/4 финала(Нептунас - Шяуляй)\n',
       '4154509 Литва. Мужчины. LKL. Плей-офф. 1/4 финала(Нептунас - Шяуляй)\n'],
      dtype=object)

Так как номера сильно отличаются, наверное, это всё же разные игры.

###Подготовка данных для предсказания по результатам четвертей

In [ ]:
temp1 = time_data.groupby(['number', 'quarter'], as_index=False) \
    .agg({feature: 'max' for feature in ['team_1', 'team_2', 'current_count', 'final_count']})
temp1.head(10) # создаём таблицу с данными по четвертям

,number,quarter,team_1,team_2,current_count,final_count
0,3986915,1.0,18,12.0,30.0,150
1,3986915,2.0,43,27.0,70.0,150
2,3986915,3.0,57,53.0,110.0,150
3,3986915,4.0,65,70.0,135.0,150
4,3986915,5.0,69,78.0,147.0,150
5,3986915,6.0,72,78.0,150.0,150
6,3986918,1.0,14,15.0,29.0,133
7,3986918,2.0,39,30.0,69.0,133
8,3986918,3.0,58,41.0,99.0,133
9,3986918,4.0,74,59.0,133.0,133


In [ ]:
temp2 = temp1.pivot_table(values=['team_1', 'team_2'], index='number', columns='quarter')
temp2.head(2) # преобразуем часть входных данных так, чтобы каждый экземпляр описывался одним вектором

team_1                          ... team_2                        
quarter    1.0   2.0   3.0   4.0   5.0  ...    2.0   3.0   4.0   5.0   6.0
number                                  ...                               
3986915   18.0  43.0  57.0  65.0  69.0  ...   27.0  53.0  70.0  78.0  78.0
3986918   14.0  39.0  58.0  74.0   NaN  ...   30.0  41.0  59.0   NaN   NaN

[2 rows x 12 columns]

In [ ]:
quarter_data = pd.concat(
    [
     temp2['team_1'][[1, 2, 3]],
     temp2['team_2'][[1, 2, 3]],
     temp1.groupby('number').agg({'final_count': 'max'})
     ], axis=1
     )
quarter_data # соединяем входные и выходные данные

,1.0,2.0,3.0,1.0,2.0,3.0,final_count
number,,,,,,,
3986915,18.0,43.0,57.0,12.0,27.0,53.0,150
3986918,14.0,39.0,58.0,15.0,30.0,41.0,133
3986921,16.0,40.0,56.0,17.0,40.0,64.0,163
3986923,14.0,19.0,36.0,16.0,19.0,41.0,163
3986924,25.0,45.0,57.0,18.0,37.0,54.0,146
...,...,...,...,...,...,...,...
4191662,22.0,29.0,48.0,17.0,39.0,58.0,149
4191737,9.0,25.0,38.0,21.0,30.0,38.0,92
4191757,19.0,42.0,56.0,20.0,36.0,48.0,135


In [ ]:
quarter_data = pd.concat([quarter_data, quarter_data.iloc[:, [3, 4, 5, 0, 1, 2, 6]]])
quarter_data # делаем аугументацию, меняя команды местами

,1.0,2.0,3.0,1.0,2.0,3.0,final_count
number,,,,,,,
3986915,18.0,43.0,57.0,12.0,27.0,53.0,150
3986918,14.0,39.0,58.0,15.0,30.0,41.0,133
3986921,16.0,40.0,56.0,17.0,40.0,64.0,163
3986923,14.0,19.0,36.0,16.0,19.0,41.0,163
3986924,25.0,45.0,57.0,18.0,37.0,54.0,146
...,...,...,...,...,...,...,...
4191662,17.0,39.0,58.0,22.0,29.0,48.0,149
4191737,21.0,30.0,38.0,9.0,25.0,38.0,92
4191757,20.0,36.0,48.0,19.0,42.0,56.0,135


##Research

###Learning

In [ ]:
for mode in ['time', 'sum', 'time_2']:
    research(time_data, mode)


Средняя ошибка на тестовой выборке: 13.77 очков.


In [ ]:
for mode in ['quarter_1', 'quarter_2', 'quarter_3', 'quarter_12', 'quarter_23', 'quarter_123']:
    research(quarter_data, mode)


Средняя ошибка на тестовой выборке: 23.6 очков.


In [ ]:
for hidden_dim in [5, 10, 100]:
    research(time_data, 'time_2', hidden_dim=hidden_dim)


Средняя ошибка на тестовой выборке: 13.46 очков.


In [ ]:
for depth in [2, 3]:
    research(time_data, 'time_2', depth=depth)


Средняя ошибка на тестовой выборке: 13.48 очков.


In [ ]:
for batch_size in [32, 512]:
    research(time_data, 'time_2', hidden_dim=100, batch_size=batch_size)


Средняя ошибка на тестовой выборке: 13.66 очков.


In [ ]:
research(time_data, 'time_2', hidden_dim=100, outlier_threshold=50)


Средняя ошибка на тестовой выборке: 13.51 очков.


In [ ]:
research(time_data, 'time_2', hidden_dim=100, depth=2)


Средняя ошибка на тестовой выборке: 13.34 очков.


In [ ]:
for activation in ['sigmoid', 'elu']:
    research(time_data, 'time_2', hidden_dim=100, depth=2, activation=activation)


Средняя ошибка на тестовой выборке: 13.53 очков.


###Results

In [ ]:
log = pd.read_csv('log.csv', index_col=0)

In [ ]:
box_plot(
    log.query('outlier_threshold == 0 & hidden_dim == 20 & depth == 1 & activation == "relu" & batch_size == 128'),
    color='mode',
    title='Влияние предобработки данных на абсолютную ошибку'
    )

Вероятно для более точного предсказания по результатам четвертей не хватает объёма выборки

In [ ]:
box_plot(
    log.query('mode == "time_2" & hidden_dim == 100 & depth == 1 & activation == "relu" & batch_size == 128'),
    color='outlier_threshold',
    title='Влияние исключения выбросов на абсолютную ошибку'
    )

Как и ожидалось, удаление выбросов из обучающей выборки не улучшает результаты на тестовой.

In [ ]:
box_plot(
    log.query('mode == "time_2" & outlier_threshold == 0 & depth == 1 & activation == "relu" & batch_size == 128'),
    color='hidden_dim',
    title='Влияние количества нейронов в скрытом слое на абсолютную ошибку'
    )

Несмотря на небольшую длину входных векторов, увеличение нейронов улучшает результат.

In [ ]:
box_plot(
    log.query('mode == "time_2" & outlier_threshold == 0 & hidden_dim == 20 & activation == "relu" & batch_size == 128'),
    color='depth',
    title='Влияние глубины сети на абсолютную ошибку'
    )

Добавить скрытые слои тоже приводит к уменьшению ошибки.

In [ ]:
box_plot(
    log.query('mode == "time_2" & outlier_threshold == 0 & hidden_dim == 100 & depth == 2 & batch_size == 128'),
    color='activation',
    title='Влияние функции активации на абсолютную ошибку'
    )

ReLU заметно выигрывает.

In [ ]:
box_plot(
    log.query('mode == "time_2" & outlier_threshold == 0 & hidden_dim == 100 & depth == 1 & activation == "relu"'),
    color='batch_size',
    title='Влияние размера пакета на абсолютную ошибку'
    )

Лучше средний размер пакета.

In [ ]:
log.groupby(['subset', 'mode', 'outlier_threshold', 'hidden_dim', 'depth', 'activation', 'batch_size']) \
    .agg({'mae': 'mean'}).loc['test'].sort_values('mae') # найдём лучшие гиперпараметры сети

mae
mode        outlier_threshold hidden_dim depth activation batch_size        
time_2      0                 100        2     relu       128         13.420
                                         1     relu       32          13.462
                                                          128         13.474
            50                100        1     relu       128         13.486
            0                 20         3     relu       128         13.515
                                         2     relu       128         13.527
                              100        1     relu       512         13.628
time        0                 20         1     relu       128         13.630
time_2      0                 20         1     relu       128         13.641
                              100        2     elu        128         13.669
sum         0                 20         1     relu       128         13.685
time_2      0                 10         1     relu       128         13.781
                              100        2     sigmoid    128         13.808
                              5          1     relu       128         13.850
quarter_3   0                 20         1     relu       128         22.986
quarter_123 0                 20         1     relu       128         22.999
quarter_12  0                 20         1     relu       128         23.154
quarter_1   0                 20         1     relu       128         23.319
quarter_23  0                 20         1     relu       128         23.383
quarter_2   0                 20         1     relu       128         24.072

#Task 2

Участник просит консультацию по диплому.


"У меня следующая идея для проекта:
У нас в Компании есть календарно-сетевые графики строительных проектов.
Сейчас происходит идентификация рисков вручную и в дальнейшем количественный анализ.
Мне бы хотелось реализовать нейронку, которая при анализе графика (виды работ, сроки, стоимость и т.п.)
распознавала бы риски проекта и отображала бы их в виде диаграммы торнадо (от наиболее вероятного до менее значительного) в объеме 10 рисков на проект.
При этом бы выводилась вероятность наступления этого риска для дальнейшей проработки графика.
Как Вы думаете это возможно или необходимо посмотреть более простую тему?"

Дайте участнику подробный ответ, описывающий подход к решению поставленной задачи.

##Ответ

На мой взгляд тема интересна, а задача вполне решаема при условии достаточного объёма размеченных данных.

На вход можно подать векторизованные параметры графика каждого проекта, а на выход вероятности каждого из возможных рисков. То есть по сути получится регрессионная модель, предсказывающая не один признак, а вектор вероятностей рисков.

Затем из предсказанных вероятностей выбираете 10 максимальных для построения диаграммы.

Возможно стоит воспользоваться PolynomialFeatures из Scikit Learn для генерации новых признаков.

Не забудьте нормализовать входные параметры.

#EOF